接續方法2剩下7個位點 進行3個為一組組合 使用5種模型預測

In [ ]:
import pandas as pd
import xgboost as xgb

In [45]:
normalized_train = pd.read_csv('../result/all_beta_normalized_train_0.35.csv')

normalized_test = pd.read_csv('../result/all_beta_normalized_test_0.35.csv')

normalized_train = normalized_train[normalized_train['Unnamed: 0'].isin(normalized_test['Unnamed: 0'])]

method2_gene=pd.read_csv("../result/overlap_select_gene_7.csv")
normalized_train_c= normalized_train[normalized_train['Unnamed: 0'].isin(method2_gene['ID'])]
normalized_train_c

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,881,882,883,884,885,886,887,888,889,890
69,cg27494647,0.213666,0.213666,0.073428,0.073428,0.161137,0.161137,0.228501,0.228501,0.150540,...,0.847160,0.847160,0.778197,0.778197,0.579254,0.579254,0.815800,0.815800,0.693585,0.693585
71,cg07792478,0.119963,0.119963,0.081919,0.081919,0.130510,0.130510,0.203574,0.203574,0.122578,...,0.864006,0.864006,0.720601,0.720601,0.853552,0.853552,0.845039,0.845039,0.304242,0.304242
76,cg24848035,0.087862,0.087862,0.034526,0.034526,0.130081,0.130081,0.138430,0.138430,0.113358,...,0.772099,0.772099,0.750224,0.750224,0.772894,0.772894,0.524201,0.524201,0.448553,0.448553
115,cg07445246,0.142992,0.142992,0.200631,0.200631,0.190305,0.190305,0.230727,0.230727,0.160928,...,0.764094,0.764094,0.739924,0.739924,0.701739,0.701739,0.752216,0.752216,0.606390,0.606390
146,cg14231297,0.122214,0.122214,0.050332,0.050332,0.092202,0.092202,0.136623,0.136623,0.099212,...,0.760635,0.760635,0.679277,0.679277,0.753650,0.753650,0.693265,0.693265,0.522578,0.522578
304,cg04574090,0.154898,0.154898,0.029502,0.029502,0.109519,0.109519,0.117190,0.117190,0.067192,...,0.839923,0.839923,0.458345,0.458345,0.738352,0.738352,0.823291,0.823291,0.531566,0.531566
359,cg20388732,0.081192,0.081192,0.049816,0.049816,0.097916,0.097916,0.096646,0.096646,0.104488,...,0.694589,0.694589,0.395435,0.395435,0.570894,0.570894,0.746251,0.746251,0.512716,0.512716


In [46]:
X_train = normalized_train_c.iloc[:, 1::2]

x = []

for i in range(445):
    X = X_train.iloc[:, i]

    X = X.values.flatten().tolist()

    x.append(X)

y = [ (0 if i < 47 else 1)  for i in range(445)]

In [54]:
from sklearn.metrics import roc_auc_score, f1_score
from itertools import combinations
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [73]:
target_labels = normalized_train_c['Unnamed: 0']
# 三個一組
combinations_list = list(combinations(target_labels, 3))

result=[]
# 定義模型參數
params = {
    'objective': 'binary:logistic',  # 二分類問題
    'eval_metric': 'logloss',  # 評估指標
    'max_depth': 3,
    'learning_rate': 1
}
models = {
    'XGBoost': xgb.XGBClassifier(**params),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='linear'),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier()
}

for model_name, model in models.items():

    # 所有可能的組合
    for combination in combinations_list:
        #print(combination)
        d1_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[0]]
        d2_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[1]]
        d3_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[2]]
    
        d1 = d1_values.iloc[:, 1::2]
        d2 = d2_values.iloc[:, 1::2]
        d3 = d3_values.iloc[:, 1::2]

        d1 = d1.values.flatten().tolist()
        d2 = d2.values.flatten().tolist()
        d3 = d3.values.flatten().tolist()

        x = []

        for i in range(445):
            x.append([d1[i],d2[i],d3[i]])
            
        # 將數據轉換為DMatrix格式
        dmatrix = xgb.DMatrix(x, label=y)

        # 進行交叉驗證，這裡使用 5-fold 交叉驗證
        cv_scores = cross_val_score(model, x, y, cv=5, scoring='accuracy')
        # 計算平均準確度
        average_accuracy = cv_scores.mean()

        # 獲取模型預測結果
        y_pred = cross_val_predict(model, x, y, cv=5)
        
        # 計算AUC
        auc_score = roc_auc_score(y, y_pred)
        
        # 計算F1 score
        f1 = f1_score(y, y_pred)
        GO_1=int(method2_gene.loc[method2_gene['ID'] == combination[0]]["GO"])
        GO_2=int(method2_gene.loc[method2_gene['ID'] == combination[1]]["GO"])
        GO_3=int(method2_gene.loc[method2_gene['ID'] == combination[2]]["GO"])
        if (GO_1!=GO_2 & GO_2!=GO_3 & GO_1!=GO_3):
            print(GO_1,GO_2,GO_3)
            if (GO_1==1 and GO_2==4 and GO_3==3):
                result.append([model_name, combination[0], combination[2], combination[1], average_accuracy, auc_score, f1])
            elif (GO_1==1 and GO_2==3 and GO_3==4):
                result.append([model_name, combination[0], combination[1], combination[2], average_accuracy, auc_score, f1])
            elif (GO_1==4 and GO_2==1 and GO_3==3):
                result.append([model_name, combination[1], combination[2], combination[0], average_accuracy, auc_score, f1])
            elif (GO_1==4 and GO_2==3 and GO_3==1):
                result.append([model_name, combination[2], combination[1], combination[0], average_accuracy, auc_score, f1])
            elif (GO_1==3 and GO_2==1 and GO_3==4):
                result.append([model_name, combination[1], combination[0], combination[2], average_accuracy, auc_score, f1])
            elif (GO_1==3 and GO_2==4 and GO_3==1):
                result.append([model_name, combination[2], combination[0], combination[1], average_accuracy, auc_score, f1])
            print(result)  
result


1 4 3
[['XGBoost', 'cg27494647', 'cg04574090', 'cg14231297', 0.9685393258426966, 0.926119961509676, 0.9823677581863981]]
1 4 4
[['XGBoost', 'cg27494647', 'cg04574090', 'cg14231297', 0.9685393258426966, 0.926119961509676, 0.9823677581863981]]
1 3 4
[['XGBoost', 'cg27494647', 'cg04574090', 'cg14231297', 0.9685393258426966, 0.926119961509676, 0.9823677581863981], ['XGBoost', 'cg27494647', 'cg04574090', 'cg20388732', 0.9640449438202247, 0.9048433657649951, 0.9798994974874372]]
1 4 3
[['XGBoost', 'cg27494647', 'cg04574090', 'cg14231297', 0.9685393258426966, 0.926119961509676, 0.9823677581863981], ['XGBoost', 'cg27494647', 'cg04574090', 'cg20388732', 0.9640449438202247, 0.9048433657649951, 0.9798994974874372], ['XGBoost', 'cg07792478', 'cg04574090', 'cg14231297', 0.9752808988764045, 0.9580348551266973, 0.9860935524652339]]
1 4 4
[['XGBoost', 'cg27494647', 'cg04574090', 'cg14231297', 0.9685393258426966, 0.926119961509676, 0.9823677581863981], ['XGBoost', 'cg27494647', 'cg04574090', 'cg2038873

[['XGBoost',
  'cg27494647',
  'cg04574090',
  'cg14231297',
  0.9685393258426966,
  0.926119961509676,
  0.9823677581863981],
 ['XGBoost',
  'cg27494647',
  'cg04574090',
  'cg20388732',
  0.9640449438202247,
  0.9048433657649951,
  0.9798994974874372],
 ['XGBoost',
  'cg07792478',
  'cg04574090',
  'cg14231297',
  0.9752808988764045,
  0.9580348551266973,
  0.9860935524652339],
 ['XGBoost',
  'cg07792478',
  'cg04574090',
  'cg20388732',
  0.9662921348314606,
  0.8967176307067252,
  0.981226533166458],
 ['XGBoost',
  'cg24848035',
  'cg04574090',
  'cg14231297',
  0.9842696629213483,
  0.9724419972201434,
  0.9911727616645649],
 ['XGBoost',
  'cg24848035',
  'cg04574090',
  'cg20388732',
  0.9617977528089888,
  0.90358708435796,
  0.9786163522012579],
 ['XGBoost',
  'cg07445246',
  'cg04574090',
  'cg14231297',
  0.9797752808988763,
  0.9511654014754625,
  0.988679245283019],
 ['XGBoost',
  'cg07445246',
  'cg04574090',
  'cg20388732',
  0.9617977528089888,
  0.8942050678926549,
  0.

In [74]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2', 'ID3', 'average_accuracy','AUC','f1_score'])
df.to_csv("../result/ml_combination_method2.csv", index=False)

挑出3個為一組最佳的組合: cg07792478,cg04574090,cg14231297 繪製normal/tumor間的關係

In [75]:
d1_values = normalized_train.loc[normalized_train['Unnamed: 0'] == 'cg07792478']
d2_values = normalized_train.loc[normalized_train['Unnamed: 0'] == 'cg04574090']
d3_values = normalized_train.loc[normalized_train['Unnamed: 0'] == 'cg14231297']


print(d1_values)

d1 = d1_values.iloc[:, 1::2]
d2 = d2_values.iloc[:, 1::2]
d3 = d3_values.iloc[:, 1::2]

d1 = d1.values.flatten().tolist()
d2 = d2.values.flatten().tolist()
d3 = d3.values.flatten().tolist()

    Unnamed: 0         1         2         3         4        5        6  \
71  cg07792478  0.119963  0.119963  0.081919  0.081919  0.13051  0.13051   

           7         8         9  ...       881       882       883       884  \
71  0.203574  0.203574  0.122578  ...  0.864006  0.864006  0.720601  0.720601   

         885       886       887       888       889       890  
71  0.853552  0.853552  0.845039  0.845039  0.304242  0.304242  

[1 rows x 891 columns]


In [77]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact

def plot_3d_scatter(elev, azim):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # 绘制 x-y 图
    ax.scatter(d1[:47], d2[:47], d3[:47], marker='o', color='blue', label='normal')
    ax.scatter(d1[47:], d2[47:], d3[47:], marker='x', color='red', label='tumor')

    # 添加标题和轴标签
    ax.set_title('3D Scatter Plot')
    ax.set_xlabel('X-cg07792478')
    ax.set_ylabel('Y-cg04574090')
    ax.set_zlabel('Z-cg14231297')

    # 添加图例
    ax.legend()

    # 调整视角
    ax.view_init(elev=elev, azim=azim)
    
    # 显示图形
    plt.show()

# 创建交互式图形
interact(plot_3d_scatter, elev=(0, 90, 5), azim=(0, 360, 5))


interactive(children=(IntSlider(value=45, description='elev', max=90, step=5), IntSlider(value=180, descriptio…

<function __main__.plot_3d_scatter(elev, azim)>

接續方法1剩下40個位點進行GO分4群後 於每群挑一個代表的位點 組合並預測